In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords

Getting volumes from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/
Getting volumes from https://humanist.kdl.kcl.ac.uk/Archives/Current/


KeyError: 'volume_dates'

In [ ]:
def clean_tokenize(text):
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]

In [ ]:
base_urls = ["https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/", "https://humanist.kdl.kcl.ac.uk/Archives/Current/"]
volume_dfs = []

In [ ]:
def scrape_humanist_data():
    for base_url in base_urls:
        print(f"Scraping volumes from {base_url}")
        try:
            response = requests.get(base_url)
            response.raise_for_status()  # Check that the request was successful
            soup = BeautifulSoup(response.text, 'html.parser')
            links = [link for link in soup.find_all('a') if link['href'].endswith('.txt')]
            
            for link in links:
                volume_url = base_url + link['href']
                volume_date = link['href'].split('.')[1]
                print(f"Fetching volume from {volume_url}")
                
                vol_response = requests.get(volume_url)
                vol_response.raise_for_status()
                text = vol_response.text
                
                volume_dfs.append({
                    'volume_text': text,
                    'volume_link': volume_url,
                    'volume_dates': volume_date,
                    'volume_number': volume_date.split('-')[0]
                })
        except requests.RequestException as e:
            print(f"Failed to download data from {base_url}: {e}")

In [ ]:
scrape_humanist_data()

Scraping volumes from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/
Failed to download data from https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/: 502 Server Error: connect failed for url: https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/
Scraping volumes from https://humanist.kdl.kcl.ac.uk/Archives/Current/
Failed to download data from https://humanist.kdl.kcl.ac.uk/Archives/Current/: 502 Server Error: connect failed for url: https://humanist.kdl.kcl.ac.uk/Archives/Current/


In [ ]:
df = pd.DataFrame(volume_dfs)

In [ ]:
df['tokens'] = df['volume_text'].apply(clean_tokenize)

KeyError: 'volume_text'

In [ ]:
df.to_csv("web_scraped_humanist_listserv_volumes.csv", index=False)